In [5]:
import re
import pandas as pd

# Your reactions as a string
reactions_str = """
- equation: H2 + M <=> H + H + M  # Reaction 1
  type: three-body
  rate-constant: {A: 4.57e+19, b: -1.4, Ea: 1.04e+05}
  efficiencies: {H2: 2.5, H2O: 12.0}
- equation: O + H2O <=> OH + OH  # Reaction 2
  rate-constant: {A: 2.95e+06, b: 2.02, Ea: 1.34e+04}
- equation: O + H2 <=> H + OH  # Reaction 3
  rate-constant: {A: 5.08e+04, b: 2.67, Ea: 6290.0}
- equation: O + O + M <=> O2 + M  # Reaction 4
  type: three-body
  rate-constant: {A: 6.17e+15, b: -0.5, Ea: 0.0}
  efficiencies: {H2: 2.5, H2O: 12.0}
- equation: H + O2 <=> O + OH  # Reaction 5
  rate-constant: {A: 3.52e+16, b: -0.7, Ea: 1.707e+04}
- equation: H + O2 (+M) <=> HO2 (+M)  # Reaction 6
  type: falloff
  low-P-rate-constant: {A: 3.5e+19, b: -1.3, Ea: 0.0}
  high-P-rate-constant: {A: 5.59e+13, b: 0.2, Ea: 0.0}
  Troe: {A: 0.7, T3: 1.0e-30, T1: 1.0e+30, T2: 1.0e+30}
  efficiencies: {H2: 2.5, H2O: 0.0, H2O2: 12.0, O2: 0.0}
  note: Hong 2010
- equation: H + O + M <=> OH + M  # Reaction 7
  type: three-body
  rate-constant: {A: 4.72e+18, b: -1.0, Ea: 0.0}
  efficiencies: {H2: 2.5, H2O: 12.0}
- equation: OH + H2 <=> H2O + H  # Reaction 8
  rate-constant: {A: 2.16e+08, b: 1.51, Ea: 3430.0}
- equation: OH + H + M <=> H2O + M  # Reaction 9
  type: three-body
  rate-constant: {A: 2.21e+22, b: -2.0, Ea: 0.0}
  efficiencies: {H2: 2.5, H2O: 12.0}
- equation: HO2 + O <=> O2 + OH  # Reaction 10
  rate-constant: {A: 1.75e+13, b: 0.0, Ea: -397.0}
- equation: HO2 + H <=> H2 + O2  # Reaction 11
  rate-constant: {A: 6.62e+13, b: 0.0, Ea: 2130.0}
- equation: HO2 + H <=> OH + OH  # Reaction 12
  rate-constant: {A: 1.69e+14, b: 0.0, Ea: 874.0}
- equation: HO2 + OH <=> H2O + O2  # Reaction 13
  rate-constant: {A: 1.9e+16, b: -1.0, Ea: 0.0}
- equation: HO2 + HO2 <=> H2O2 + O2  # Reaction 14
  rate-constant: {A: 4.2e+14, b: 0.0, Ea: 1.198e+04}
- equation: H2O2 (+M) <=> OH + OH (+M)  # Reaction 15
  type: falloff
  low-P-rate-constant: {A: 1.2e+17, b: 0.0, Ea: 4.55e+04}
  high-P-rate-constant: {A: 3.0e+14, b: 0.0, Ea: 4.846e+04}
  Troe: {A: 0.5, T3: 1.0e-90, T1: 1.0e+90}
- equation: H2O2 + O <=> OH + HO2  # Reaction 16
  rate-constant: {A: 9.64e+06, b: 2.0, Ea: 3970.0}
- equation: H2O2 + H <=> H2O + OH  # Reaction 17
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 3590.0}
- equation: H2O2 + H <=> HO2 + H2  # Reaction 18
  rate-constant: {A: 4.82e+13, b: 0.0, Ea: 7950.0}
- equation: H2O2 + OH <=> H2O + HO2  # Reaction 19
  rate-constant: {A: 5.802e+14, b: 0.0, Ea: 9557.0}
- equation: H + 2 O2 <=> HO2 + O2  # Reaction 20
  rate-constant: {A: 3.0e+20, b: -1.72, Ea: 0.0}
  note: Meynet
- equation: H + O2 + H2O <=> HO2 + H2O  # Reaction 21
  rate-constant: {A: 9.38e+18, b: -0.76, Ea: 0.0}
  note: Meynet
# - equation: 2 H + H2 <=> 2 H2  # Reaction 22
#   rate-constant: {A: 9.0e+16, b: -0.6, Ea: 0.0}
#   note: Meynet
# - equation: 2 H + H2O <=> H2 + H2O  # Reaction 23
#   rate-constant: {A: 6.0e+19, b: -1.25, Ea: 0.0}
#   note: Meynet
# - equation: H + HO2 <=> O + H2O  # Reaction 24
#   rate-constant: {A: 3.97e+12, b: 0.0, Ea: 671.0}
#   note: Meynet
- equation: N + H2 <=> H + NH  # Reaction 25
  rate-constant: {A: 1.6e+14, b: 0.0, Ea: 2.514e+04}
- equation: N + O2 <=> NO + O  # Reaction 26
  rate-constant: {A: 6.4e+09, b: 1.0, Ea: 6280.0}
- equation: N + OH <=> NO + H  # Reaction 27
  rate-constant: {A: 3.8e+13, b: 0.0, Ea: 0.0}
- equation: N + HO2 <=> NH + O2  # Reaction 28
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 2000.0}
- equation: N + HO2 <=> NO + OH  # Reaction 29
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 2000.0}
- equation: N + NO <=> N2 + O  # Reaction 30
  rate-constant: {A: 3.27e+12, b: 0.3, Ea: 0.0}
- equation: N + NO2 <=> NO + NO  # Reaction 31
  rate-constant: {A: 4.0e+12, b: 0.0, Ea: 0.0}
- equation: N + NO2 <=> N2O + O  # Reaction 32
  rate-constant: {A: 5.0e+12, b: 0.0, Ea: 0.0}
- equation: N + NO2 <=> N2 + O2  # Reaction 33
  rate-constant: {A: 1.0e+12, b: 0.0, Ea: 0.0}
- equation: N + HNO <=> NH + NO  # Reaction 34
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 2000.0}
- equation: N + HNO <=> N2O + H  # Reaction 35
  rate-constant: {A: 5.0e+10, b: 0.5, Ea: 3000.0}
- equation: N + N2O <=> N2 + NO  # Reaction 36
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 1.987e+04}
- equation: NO + M <=> N + O + M  # Reaction 37
  type: three-body
  rate-constant: {A: 9.64e+14, b: 0.0, Ea: 1.484e+05}
  efficiencies: {N2: 1.5, H2O: 10.0}
- equation: NO + H2 <=> HNO + H  # Reaction 38
  rate-constant: {A: 1.39e+13, b: 0.0, Ea: 5.653e+04}
  note: CO2/2.50/
- equation: NO + O (+M) <=> NO2 (+M)  # Reaction 39
  type: falloff
  low-P-rate-constant: {A: 4.72e+24, b: -2.87, Ea: 1551.0}
  high-P-rate-constant: {A: 1.3e+15, b: -0.75, Ea: 0.0}
  Troe: {A: 0.957, T3: 1.0e-90, T1: 8332.0}
- equation: NO + H (+M) <=> HNO (+M)  # Reaction 40
  type: falloff
  low-P-rate-constant: {A: 8.96e+19, b: -1.32, Ea: 735.2}
  high-P-rate-constant: {A: 1.52e+15, b: -0.41, Ea: 0.0}
- equation: NO + OH (+M) <=> HONO (+M)  # Reaction 41
  type: falloff
  low-P-rate-constant: {A: 5.08e+23, b: -2.51, Ea: -67.56}
  high-P-rate-constant: {A: 1.99e+12, b: -0.05, Ea: -721.0}
  Troe: {A: 0.62, T3: 1.0e-90, T1: 1.0e+90}
  efficiencies: {H2O: 5.0}
- equation: NO + OH <=> HONO  # Reaction 42
  rate-constant: {A: 2.64e+12, b: -0.085, Ea: -647.0}
  note: Meynet
- equation: NO2 + O (+M) <=> NO3 (+M)  # Reaction 43
  type: falloff
  low-P-rate-constant: {A: 1.49e+28, b: -4.08, Ea: 2467.0}
  high-P-rate-constant: {A: 1.33e+13, b: 0.0, Ea: 0.0}
  Troe: {A: 0.826, T3: 1.0e-90, T1: 3191.0}
- equation: NO2 + H <=> NO + OH  # Reaction 44
  rate-constant: {A: 1.32e+14, b: 0.0, Ea: 361.6}
  note: Yetter
- equation: NO2 + OH (+M) <=> HNO3 (+M)  # Reaction 45
  type: falloff
  low-P-rate-constant: {A: 6.42e+32, b: -5.49, Ea: 2350.0}
  high-P-rate-constant: {A: 2.41e+13, b: 0.0, Ea: 0.0}
  Troe: {A: 0.837, T3: 1.0e-90, T1: 1657.0}
  note: NO2+H=NO+OH                                 2.500E+14   0.00    6.760E+02   !
    Baulch 2005 A/5.000E+14 error (+/-0.1-0.3) 230-800 K 2.500E+14
- equation: HO2 + NO <=> NO2 + OH  # Reaction 46
  rate-constant: {A: 2.11e+12, b: 0.0, Ea: -479.0}
- equation: NO2 + NO2 <=> NO3 + NO  # Reaction 47
  rate-constant: {A: 9.64e+09, b: 0.73, Ea: 2.092e+04}
- equation: NO2 + NO2 <=> NO + NO + O2  # Reaction 48
  rate-constant: {A: 1.63e+12, b: 0.0, Ea: 2.612e+04}
- equation: NO2 + O <=> O2 + NO  # Reaction 49
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 600.0}
- equation: NO3 + H <=> NO2 + OH  # Reaction 50
  rate-constant: {A: 6.0e+13, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: NO3 + O <=> NO2 + O2  # Reaction 51
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: NO3 + OH <=> NO2 + HO2  # Reaction 52
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: NO3 + HO2 <=> NO2 + O2 + OH  # Reaction 53
  rate-constant: {A: 1.5e+12, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: NO3 + NO2 <=> NO + NO2 + O2  # Reaction 54
  rate-constant: {A: 4.9e+10, b: 0.0, Ea: 2940.0}
  note: Meynet
- equation: N2O (+M) <=> N2 + O (+M)  # Reaction 55
  type: falloff
  low-P-rate-constant: {A: 8.7888e+14, b: 0.0, Ea: 5.755e+04}
  high-P-rate-constant: {A: 7.91e+10, b: 0.0, Ea: 5.604e+04}
  efficiencies: {H2O: 7.5, NO: 2.0}
- equation: N2O + O <=> O2 + N2  # Reaction 56
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 2.8e+04}
- equation: N2O + O <=> NO + NO  # Reaction 57
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 2.8e+04}
- equation: N2O + H <=> N2 + OH  # Reaction 58
  rate-constant: {A: 2.23e+14, b: 0.0, Ea: 1.675e+04}
- equation: N2O + OH <=> HO2 + N2  # Reaction 59
  rate-constant: {A: 2.0e+12, b: 0.0, Ea: 4.0e+04}
- equation: NH + M <=> N + H + M  # Reaction 60
  type: three-body
  rate-constant: {A: 2.65e+14, b: 0.0, Ea: 7.551e+04}
- equation: NH + O2 <=> HNO + O  # Reaction 61
  rate-constant: {A: 4.6e+05, b: 2.0, Ea: 6500.0}
  note: KLIMIC2011 cited Skreiberg 2004
- equation: NH + O2 <=> NO + OH  # Reaction 62
  rate-constant: {A: 7.6e+10, b: 0.0, Ea: 1530.0}
- equation: NH + O <=> NO + H  # Reaction 63
  rate-constant: {A: 5.5e+13, b: 0.0, Ea: 0.0}
- equation: NH + O <=> N + OH  # Reaction 64
  rate-constant: {A: 3.72e+13, b: 0.0, Ea: 0.0}
- equation: NH + OH <=> HNO + H  # Reaction 65
  rate-constant: {A: 2.0e+13, b: 0.0, Ea: 0.0}
- equation: NH + OH <=> N + H2O  # Reaction 66
  rate-constant: {A: 5.0e+11, b: 0.5, Ea: 2000.0}
- equation: NH + N <=> N2 + H  # Reaction 67
  rate-constant: {A: 3.0e+13, b: 0.0, Ea: 0.0}
- equation: NH + NO <=> N2O + H  # Reaction 68
  rate-constant: {A: 2.94e+14, b: -0.4, Ea: 0.0}
- equation: NH + NO <=> N2 + OH  # Reaction 69
  rate-constant: {A: 2.16e+13, b: -0.23, Ea: 0.0}
- equation: NH + NO2 <=> NO + HNO  # Reaction 70
  rate-constant: {A: 1.0e+11, b: 0.5, Ea: 4000.0}
  note: ??? -0.5
- equation: NH + NO2 <=> N2O + OH  # Reaction 71
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 0.0}
- equation: NH + NH <=> N2 + H + H  # Reaction 72
  rate-constant: {A: 5.1e+13, b: 0.0, Ea: 0.0}
- equation: NH + NH2 <=> N2H2 + H  # Reaction 73
  rate-constant: {A: 3.16e+13, b: 0.0, Ea: 994.0}
  note: Meynet
- equation: NH + NH <=> NNH + H  # Reaction 74
  rate-constant: {A: 7.94e+11, b: 0.5, Ea: 994.0}
  note: Meynet
- equation: NH + NH <=> NH2 + N  # Reaction 75
  rate-constant: {A: 2.0e+11, b: 0.5, Ea: 1987.0}
  note: Meynet
- equation: NH2 + O2 <=> HNO + OH  # Reaction 76
  rate-constant: {A: 6.2e+07, b: 1.2, Ea: 3.51e+04}
  note: Skreiberg 2004 also used by Zhang 2011 /Lamoureux 2016
- equation: NH2 + O <=> HNO + H  # Reaction 77
  rate-constant: {A: 6.63e+14, b: -0.5, Ea: 0.0}
- equation: NH2 + O <=> NH + OH  # Reaction 78
  rate-constant: {A: 6.75e+12, b: 0.0, Ea: 0.0}
- equation: NH2 + H <=> NH + H2  # Reaction 79
  rate-constant: {A: 6.92e+13, b: 0.0, Ea: 3650.0}
- equation: NH2 + OH <=> NH + H2O  # Reaction 80
  rate-constant: {A: 4.0e+06, b: 2.0, Ea: 1000.0}
- equation: NH2 + N <=> N2 + H + H  # Reaction 81
  rate-constant: {A: 7.2e+13, b: 0.0, Ea: 0.0}
- equation: NH2 + NO <=> NNH + OH  # Reaction 82
  rate-constant: {A: 2.8e+13, b: -0.55, Ea: 0.0}
- equation: NH2 + NO <=> N2 + H2O  # Reaction 83
  rate-constant: {A: 8.82e+15, b: -1.25, Ea: 0.0}
- equation: NH2 + NO <=> N2O + H2  # Reaction 84
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 2.464e+04}
- equation: NH2 + NO <=> HNO + NH  # Reaction 85
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 4.0e+04}
- equation: NH2 + NO2 <=> N2O + H2O  # Reaction 86
  rate-constant: {A: 3.28e+18, b: -2.2, Ea: 0.0}
- equation: NH2 + NH2 <=> NH3 + NH  # Reaction 87
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 1.0e+04}
- equation: NH2 + M <=> NH + H + M  # Reaction 88
  type: three-body
  rate-constant: {A: 3.16e+23, b: -2.0, Ea: 9.14e+04}
  note: Meynet
- equation: NH2 + NO <=> N2 + H + OH  # Reaction 89
  rate-constant: {A: 6.3e+19, b: -2.5, Ea: 1900.0}
  note: Meynet
- equation: NH2 + NH2 <=> N2H2 + H2  # Reaction 90
  rate-constant: {A: 3.98e+13, b: 0.0, Ea: 1.192e+04}
  note: Meynet
- equation: NH2 + NO2 <=> H2NO + NO  # Reaction 91
  rate-constant: {A: 1.05e+13, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: NH3 (+M) <=> NH2 + H (+M)  # Reaction 92
  type: falloff
  low-P-rate-constant: {A: 2.0e+16, b: 0.0, Ea: 9.315e+04}
  high-P-rate-constant: {A: 9.0e+16, b: -0.39, Ea: 1.103e+05}
  Troe: {A: 0.42, T3: 4581.0, T1: 102.0, T2: 1.0e+14}
  efficiencies: {H2: 1.0, H2O: 6.4, O2: 0.45, N2: 0.4}
  note: Baulch 2005 A/2.800E+17 error (+/-0.5) 2000- 3000 K
- equation: NH3 + O <=> NH2 + OH  # Reaction 93
  rate-constant: {A: 9.4e+06, b: 1.94, Ea: 6460.0}
  note: |-
    Baulch 2005 A/1.820E+16 error(+/-0.3) 2000-3000 K M=AR
    Griffith & Barnard
- equation: NH3 + H <=> NH2 + H2  # Reaction 94
  rate-constant: {A: 6.4e+05, b: 2.39, Ea: 1.017e+04}
- equation: NH3 + OH <=> NH2 + H2O  # Reaction 95
  rate-constant: {A: 2.04e+06, b: 2.04, Ea: 566.0}
- equation: NH3 + HO2 <=> NH2 + H2O2  # Reaction 96
  rate-constant: {A: 3.0e+11, b: 0.0, Ea: 2.2e+04}
- equation: NH2 + HO2 <=> NH3 + O2  # Reaction 97
  rate-constant: {A: 9.2e+05, b: 1.94, Ea: -1152.0}
  note: Skreiberg 2004 also used by KLIMIC2011
- equation: NH3 + M <=> NH + H2 + M  # Reaction 98
  type: three-body
  rate-constant: {A: 6.3e+14, b: 0.0, Ea: 9.339e+04}
  note: Meynet
- equation: NH3 + NH2 <=> N2H3 + H2  # Reaction 99
  rate-constant: {A: 1.0e+11, b: 0.5, Ea: 2.16e+04}
  note: Coppens 2007
- equation: NNH + M <=> N2 + H + M  # Reaction 100
  type: three-body
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 3000.0}
- equation: NNH + H <=> N2 + H2  # Reaction 101
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 0.0}
- equation: NNH + NO <=> N2 + HNO  # Reaction 102
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 0.0}
- equation: NNH + O <=> N2O + H  # Reaction 103
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 0.0}
- equation: NNH + OH <=> N2 + H2O  # Reaction 104
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 0.0}
- equation: NNH + NH <=> N2 + NH2  # Reaction 105
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 0.0}
- equation: NNH + NH2 <=> N2 + NH3  # Reaction 106
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 0.0}
- equation: HNO + O <=> OH + NO  # Reaction 107
  rate-constant: {A: 1.81e+13, b: 0.0, Ea: 0.0}
- equation: N2H2 + M <=> NNH + H + M  # Reaction 108
  type: three-body
  rate-constant: {A: 1.0e+16, b: 0.0, Ea: 4.968e+04}
  note: |-
    NNH=N2+H                                   3.26E+11     -.961         4222.0 ! Meynet
    NNH+O2=HO2+N2                             .500E+13      .00000E+00   .00000E+00 ! Meynet
    NNH+O=OH+N2                               .250E+14      .00000E+00   .00000E+00 ! Meynet
    NNH+O=NH+NO                               .700E+14      .00000E+00   .00000E+00 ! Meynet
    NNH+M=NH+N+M                               1.00E+15  0.00E+00  6.954E+04 ! Meynet
    NNH+N=NH+N2                                3.16E+13  0.00E+00  1.987E+03 ! Meynet
    NNH+NNH=N2H2+N2                            1.00E+13  0.00E+00  9.935E+03 ! Meynet
    Meynet
- equation: N2H2 + M <=> NH + NH + M  # Reaction 109
  type: three-body
  rate-constant: {A: 3.16e+16, b: 0.0, Ea: 9.935e+04}
  note: Meynet
- equation: N2H2 + H <=> NNH + H2  # Reaction 110
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 994.0}
  note: Meynet
- equation: N2H2 + O <=> NNH + OH  # Reaction 111
  rate-constant: {A: 1.0e+11, b: 0.5, Ea: 0.0}
  note: Meynet
- equation: N2H2 + OH <=> NNH + H2O  # Reaction 112
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 1987.0}
  note: Meynet
- equation: N2H2 + NH <=> NNH + NH2  # Reaction 113
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 994.0}
  note: Meynet
- equation: N2H2 + NH2 <=> NNH + NH3  # Reaction 114
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 3974.0}
  note: Meynet
- equation: N2H2 + NO <=> N2O + NH2  # Reaction 115
  rate-constant: {A: 3.0e+12, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: N2H2 + NH2 <=> N2H3 + NH  # Reaction 116
  rate-constant: {A: 1.0e+11, b: 0.5, Ea: 3.378e+04}
  note: Meynet
- equation: N2H2 + N2H2 <=> N2H3 + NNH  # Reaction 117
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 9935.0}
  note: Meynet
- equation: N2H3 + M <=> NH2 + NH + M  # Reaction 118
  type: three-body
  rate-constant: {A: 1.0e+16, b: 0.0, Ea: 4.173e+04}
  note: Meynet
- equation: N2H3 + H <=> N2H2 + H2  # Reaction 119
  rate-constant: {A: 1.0e+12, b: 0.0, Ea: 1987.0}
  note: Meynet
- equation: N2H3 + H <=> NH2 + NH2  # Reaction 120
  rate-constant: {A: 1.58e+12, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: N2H3 + H <=> NH + NH3  # Reaction 121
  rate-constant: {A: 1.0e+11, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: N2H3 + O <=> N2H2 + OH  # Reaction 122
  rate-constant: {A: 3.16e+11, b: 0.5, Ea: 0.0}
  note: Meynet
- equation: N2H3 + O <=> NNH + H2O  # Reaction 123
  rate-constant: {A: 3.16e+11, b: 0.5, Ea: 0.0}
  note: Meynet
- equation: N2H3 + OH <=> N2H2 + H2O  # Reaction 124
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 1987.0}
  note: Meynet
- equation: N2H3 + NH2 <=> N2H2 + NH3  # Reaction 125
  rate-constant: {A: 1.0e+11, b: 0.5, Ea: 0.0}
  note: Meynet
- equation: N2H3 + N2H2 <=> N2H4 + NNH  # Reaction 126
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 9935.0}
  note: Meynet
- equation: N2H4 + M <=> NH2 + NH2 + M  # Reaction 127
  type: three-body
  rate-constant: {A: 4.0e+15, b: 0.0, Ea: 4.093e+04}
  note: Meynet
- equation: N2H4 + M <=> N2H3 + H + M  # Reaction 128
  type: three-body
  rate-constant: {A: 1.0e+15, b: 0.0, Ea: 6.358e+04}
  note: Meynet
- equation: N2H4 + H <=> N2H3 + H2  # Reaction 129
  rate-constant: {A: 1.29e+13, b: 0.0, Ea: 2503.0}
  note: Meynet
- equation: N2H4 + H <=> NH2 + NH3  # Reaction 130
  rate-constant: {A: 4.46e+09, b: 0.0, Ea: 3099.0}
  note: Meynet
- equation: N2H4 + O <=> N2H2 + H2O  # Reaction 131
  rate-constant: {A: 6.31e+13, b: 0.0, Ea: 1192.0}
  note: Meynet
- equation: N2H4 + O <=> N2H3 + OH  # Reaction 132
  rate-constant: {A: 2.51e+12, b: 0.0, Ea: 1192.0}
  note: Meynet
- equation: N2H4 + OH <=> N2H3 + H2O  # Reaction 133
  rate-constant: {A: 3.98e+13, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: N2H4 + NH <=> NH2 + N2H3  # Reaction 134
  rate-constant: {A: 1.0e+12, b: 0.0, Ea: 1987.0}
  note: Meynet
- equation: N2H4 + NH2 <=> N2H3 + NH3  # Reaction 135
  rate-constant: {A: 3.98e+11, b: 0.5, Ea: 1987.0}
  note: Meynet
- equation: N2H4 + N2H2 <=> N2H3 + N2H3  # Reaction 136
  rate-constant: {A: 2.5e+10, b: 0.5, Ea: 2.981e+04}
  note: Meynet
- equation: HNO + OH <=> H2O + NO  # Reaction 137
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 993.5}
- equation: HNO + NO <=> N2O + OH  # Reaction 138
  rate-constant: {A: 2.0e+12, b: 0.0, Ea: 2.6e+04}
- equation: HNO + NO2 <=> HONO + NO  # Reaction 139
  rate-constant: {A: 6.02e+11, b: 0.0, Ea: 1987.0}
- equation: HNO + HNO <=> H2O + N2O  # Reaction 140
  rate-constant: {A: 8.51e+08, b: 0.0, Ea: 3080.0}
- equation: HNO + O2 <=> NO + HO2  # Reaction 141
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 2.5e+04}
- equation: HNO + NH2 <=> NO + NH3  # Reaction 142
  rate-constant: {A: 2.0e+13, b: 0.0, Ea: 1000.0}
- equation: O2 + HNO <=> NO2 + OH  # Reaction 143
  rate-constant: {A: 1.5e+13, b: 0.0, Ea: 1.0e+04}
  note: Meynet
- equation: H2NO + H <=> HNO + H2  # Reaction 144
  rate-constant: {A: 3.0e+07, b: 2.0, Ea: 2000.0}
  note: Meynet
- equation: H2NO + H <=> NH2 + OH  # Reaction 145
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 0.0}
  note: Meynet
- equation: H2NO + O <=> HNO + OH  # Reaction 146
  rate-constant: {A: 3.0e+07, b: 2.0, Ea: 2000.0}
  note: Meynet
- equation: H2NO + OH <=> HNO + H2O  # Reaction 147
  rate-constant: {A: 2.0e+07, b: 2.0, Ea: 1000.0}
  note: Meynet
- equation: H2NO + NO2 <=> HNO + HONO  # Reaction 148
  rate-constant: {A: 6.0e+11, b: 0.0, Ea: 2000.0}
  note: Meynet
- equation: H2NO + NH2 <=> HNO + NH3  # Reaction 149
  rate-constant: {A: 3.0e+12, b: 0.0, Ea: 1000.0}
  note: Meynet
- equation: HONO + O <=> OH + NO2  # Reaction 150
  rate-constant: {A: 1.2e+13, b: 0.0, Ea: 5961.0}
- equation: HONO + H <=> H2 + NO2  # Reaction 151
  rate-constant: {A: 1.2e+13, b: 0.0, Ea: 7352.0}
- equation: HONO + OH <=> H2O + NO2  # Reaction 152
  rate-constant: {A: 1.26e+10, b: 1.0, Ea: 135.1}
- equation: CL + HO2 <=> HCL + O2  # Reaction 153
  rate-constant: {A: 1.8e+13, b: 0.0, Ea: 0.0}
  note: |-
    HONO+NH=NH2+NO2               		   1.00E+13  0.00E+00  0.000E+00 ! Meynet
    HONO+NH2=NH3+NO2                           5.00E+12  0.00E+00  0.000E+00 ! Meynet
    HONO+HONO=NO+NO2+H2O                       2.30E+12  0.00E+00  8.400E+03 ! Meynet
- equation: CL + NH2 <=> HCL + NH  # Reaction 154
  rate-constant: {A: 5.01e+10, b: 0.5, Ea: 0.0}
- equation: CL + O2 + M <=> CLOO + M  # Reaction 155
  type: three-body
  rate-constant: {A: 9.7e+11, b: 0.0, Ea: 0.0}
- equation: CL + O2 + M <=> CLO2 + M  # Reaction 156
  type: three-body
  rate-constant: {A: 8.0e+06, b: 0.0, Ea: 5200.0}
- equation: CL + N2O <=> CLO + N2  # Reaction 157
  rate-constant: {A: 1.2e+14, b: 0.0, Ea: 3.35e+04}
- equation: NH3 + CL <=> HCL + NH2  # Reaction 158
  rate-constant: {A: 4.47e+11, b: 0.5, Ea: 100.0}
- equation: CL + CL + M <=> CL2 + M  # Reaction 159
  type: three-body
  rate-constant: {A: 2.3e+19, b: -1.5, Ea: 0.0}
  efficiencies: {H2: 2.0, CL2: 6.9, N2: 2.0}
  note: Pelucchi
- equation: CLOO + CL <=> CLO + CLO  # Reaction 160
  rate-constant: {A: 4.8e+12, b: 0.0, Ea: 0.0}
- equation: CL2 + NO <=> NOCL + CL  # Reaction 161
  rate-constant: {A: 2.7e+12, b: 0.0, Ea: 1.99e+04}
- equation: CL2 + H <=> HCL + CL  # Reaction 162
  rate-constant: {A: 8.6e+13, b: 0.0, Ea: 1172.0}
  note: Baulch et al., 1981, Pelucchi
- equation: CL + H2 <=> HCL + H  # Reaction 163
  rate-constant: {A: 9.5e+07, b: 1.72, Ea: 3060.0}
  note: Kumaran et al., 1994 ! Pelucchi
- equation: H + CL + M <=> HCL + M  # Reaction 164
  type: three-body
  rate-constant: {A: 2.0e+23, b: -2.45, Ea: 0.0}
  efficiencies: {H2: 2.0, CL2: 2.0, N2: 2.0}
  note: Schading and Roth, 1994 (Pelucchi)
- equation: CL + H2O2 <=> HCL + HO2  # Reaction 165
  rate-constant: {A: 6.6e+12, b: 0.0, Ea: 1950.0}
  note: Atkinson et al., 2007 (Pelucchi)
- equation: CL + HO2 <=> CLO + OH  # Reaction 166
  rate-constant: {A: 3.8e+13, b: 0.0, Ea: 1200.0}
  note: Atkinson et al., 2007 (Pelucchi)
- equation: CL2 + O <=> CL + CLO  # Reaction 167
  rate-constant: {A: 4.5e+12, b: 0.0, Ea: 3279.0}
  note: Wine et al., 1985 (Pelucchi)
- equation: CL2 + OH <=> HOCL + CL  # Reaction 168
  rate-constant: {A: 2.2e+08, b: 1.35, Ea: 1480.0}
  note: Bryukov et al., 2004 (Pelucchi)
- equation: CLO + NO <=> CL + NO2  # Reaction 169
  rate-constant: {A: 6.78e+12, b: 0.0, Ea: 311.0}
- equation: CLO + HNO <=> HCL + NO2  # Reaction 170
  rate-constant: {A: 3.0e+12, b: 0.0, Ea: 0.0}
- equation: CLO + NH2 <=> HNO + HCL  # Reaction 171
  rate-constant: {A: 2.5e+12, b: 0.0, Ea: 0.0}
- equation: CLO + CLO <=> CL2 + O2  # Reaction 172
  rate-constant: {A: 1.0e+11, b: 0.0, Ea: 0.0}
- equation: CLO + O <=> CL + O2  # Reaction 173
  rate-constant: {A: 6.6e+13, b: 0.0, Ea: 440.0}
- equation: CLO + H2 <=> CLOH + H  # Reaction 174
  rate-constant: {A: 6.03e+11, b: 0.0, Ea: 1.41e+04}
- equation: CLO + NH3 <=> NH2 + CLOH  # Reaction 175
  rate-constant: {A: 6.0e+11, b: 0.5, Ea: 6400.0}
- equation: CLO + H <=> CL + OH  # Reaction 176
  rate-constant: {A: 3.8e+13, b: 0.0, Ea: 0.0}
  note: Wategaonkar and Setser, 1989 (Pelucchi)
- equation: CLO + H <=> HCL + O  # Reaction 177
  rate-constant: {A: 8.4e+12, b: 0.0, Ea: 0.0}
  note: Wategaonkar and Setser, 1989 (Pelucchi)
- equation: CLO + OH <=> HCL + O2  # Reaction 178
  rate-constant: {A: 3.5e+05, b: 1.67, Ea: -3827.0}
  note: Zhu and Lin, 2011 (Pelucchi)
- equation: CLO + HO2 <=> HOCL + O2  # Reaction 179
  rate-constant: {A: 7800.0, b: 2.4, Ea: 5110.0}
  note: Zhu and Lin, 2011 (Pelucchi)
- equation: CLO + HO2 <=> CLOO + OH  # Reaction 180
  rate-constant: {A: 4.6e+05, b: 1.8, Ea: 2116.0}
  note: Zhu and Lin, 2011 (Pelucchi)
- equation: CLO2 + CLO2 <=> CLO + CLO3  # Reaction 181
  rate-constant: {A: 1.82e+13, b: 0.0, Ea: 1.8e+04}
- equation: CLO2 + NO <=> CLO + NO2  # Reaction 182
  rate-constant: {A: 1.0e+11, b: 0.0, Ea: 0.0}
- equation: CLO2 + NH <=> CLO + HNO  # Reaction 183
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 0.0}
- equation: CLO2 + CL <=> 2 CLO  # Reaction 184
  rate-constant: {A: 5.0e+13, b: 0.0, Ea: 6000.0}
- equation: CLO3 <=> CLO + O2  # Reaction 185
  rate-constant: {A: 1.7e+13, b: 0.5, Ea: 0.0}
- equation: CLOH + O <=> HCL + O2  # Reaction 186
  rate-constant: {A: 1.2e+14, b: 0.0, Ea: 0.0}
- equation: CLOH + OH <=> CLO + H2O  # Reaction 187
  rate-constant: {A: 1.82e+13, b: 0.0, Ea: 0.0}
- equation: CLOH + HCL <=> H2O + CL2  # Reaction 188
  rate-constant: {A: 4.0e+11, b: 0.0, Ea: 1.0e+04}
- equation: CLOH + CLO <=> CL2 + HO2  # Reaction 189
  rate-constant: {A: 1.0e+11, b: 0.0, Ea: 1.0e+04}
- equation: CLOH + CLO <=> CLOO + HCL  # Reaction 190
  rate-constant: {A: 4.0e+10, b: 0.0, Ea: 0.0}
- equation: CLOH + NH <=> NOCL + H2  # Reaction 191
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 0.0}
- equation: CLOH + HNO <=> NOCL + H2O  # Reaction 192
  rate-constant: {A: 3.0e+12, b: 0.0, Ea: 0.0}
- equation: NOCL + M <=> NO + CL + M  # Reaction 193
  type: three-body
  rate-constant: {A: 2.0e+17, b: 0.0, Ea: 3.77e+04}
- equation: NOCL + CLO <=> NO2 + CL2  # Reaction 194
  rate-constant: {A: 1.5e+12, b: 0.0, Ea: 0.0}
- equation: HCL + HO2 <=> CLO + H2O  # Reaction 195
  rate-constant: {A: 3.0e+12, b: 0.0, Ea: 0.0}
- equation: HCL + O <=> CL + OH  # Reaction 196
  rate-constant: {A: 5.9e+05, b: 2.11, Ea: 4024.0}
- equation: HCL + OH <=> H2O + CL  # Reaction 197
  rate-constant: {A: 5.0e+11, b: 0.0, Ea: 750.0}
- equation: HCLO4 <=> OH + CLO3  # Reaction 198
  rate-constant: {A: 1.0e+13, b: 0.0, Ea: 3.91e+04}
- equation: HCLO4 + NO <=> CLO + NO2 + HO2  # Reaction 199
  rate-constant: {A: 5.0e+14, b: 0.0, Ea: 1.0e+04}
- equation: HCLO4 + HNO <=> H2O + NO + CLO3  # Reaction 200
  rate-constant: {A: 1.5e+13, b: 0.0, Ea: 6000.0}
- equation: HCLO4 + HNO <=> H2O + NO2 + CLO2  # Reaction 201
  rate-constant: {A: 2.0e+13, b: 0.0, Ea: 6000.0}
- equation: HCLO4 + NH <=> CLO2 + HNO + OH  # Reaction 202
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 0.0}
- equation: HCLO4 + NH <=> CLO3 + N + H2O  # Reaction 203
  rate-constant: {A: 1.0e+14, b: 0.0, Ea: 1.1e+04}
- equation: HCLO4 + NH2 <=> H2O + HNO + CLO2  # Reaction 204
  rate-constant: {A: 1.0e+12, b: 0.0, Ea: 0.0}
- equation: HCLO4 + NH2 <=> CLOH + HNO + HO2  # Reaction 205
  rate-constant: {A: 1.0e+11, b: 0.0, Ea: 0.0}
- equation: 1.62 H2O + 1.015 O2 + 0.265 N2 + 0.12 N2O + 0.23 NO + 0.76 HCL + 0.12 CL2 + 0.523 C2H4 <=> 0.469 CO + 0.577 CO2 + 0.093 CL + 0.034 H + 0.907 HCL + 0.417 H2 + 2.089 H2O + 0.48 N2 + 0.04 NO + 0.009 O + 0.139 OH + 0.05 O2
  rate-constant: {A: 1.938e+11, b: 0.0, Ea: 30000}
"""

# Create a list to store reaction data
reactions_data = []

# Regular expression to match equation, Ea, b, and A
pattern = r'- equation: (.*?)\n\s+rate-constant: {A: (.*?), b: (.*?), Ea: (.*?)}'

# Find all matches in the string
matches = re.findall(pattern, reactions_str)

for match in matches:
    equation, A, b, Ea = match
    head, sep, tail = equation.partition('#')
    reactions_data.append({
        "Equation": head.strip(),
        "Ea": float(Ea),
        "b": float(b),
        "A": float(A),
    })

# Create a DataFrame
df = pd.DataFrame(reactions_data)

# Export the DataFrame to an Excel file
df.to_excel("reactions_table.xlsx", index=False, engine='openpyxl')


In [3]:
import pandas as pd

# Read the Excel file
input_file = 'reactions_table.xlsx'
df = pd.read_excel(input_file, engine='openpyxl')

# Remove the "# Reaction" number from the first column
df['Equation'] = df['Equation'].str.replace(r'# Reaction \d+ ', '', regex=True)

# Save the modified data to a new Excel file
output_file = 'modified_reaction_table.xlsx'
df.to_excel(output_file, index=False, engine='openpyxl')

print(f'Modified data exported to {output_file}')


Modified data exported to modified_reaction_table.xlsx
